In [28]:
import sys    
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
import torch as T
import torch.nn as nn 

In [29]:
test_state= T.tensor([[-1.4355, -0.7806,  0.3042,  1.1601]])
test_action= T.tensor([[0.8233, 0.8126]])

In [30]:
test_critic1= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic1', learning_rate=0.02)
# test_critic1.save_checkpoint()
print(test_critic1(test_state, test_action))
test_critic1.load_checkpoint()
print(test_critic1(test_state, test_action))

tensor([[-0.1939]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[-0.4471]], grad_fn=<AddmmBackward0>)


In [31]:
test_critic2= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic2', learning_rate=0.02)

print(test_critic2(test_state, test_action))
for target_param, param in zip(test_critic2.parameters(), test_critic1.parameters()):
    target_param.data.copy_(param.data)
print(test_critic2(test_state, test_action))

tensor([[-0.8048]], grad_fn=<AddmmBackward0>)
tensor([[-0.4471]], grad_fn=<AddmmBackward0>)


In [32]:
test_actor1= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), 'test_actor', 0.02)
# test_actor1.save_checkpoint()
print(test_actor1(test_state))
test_actor1.load_checkpoint()
print(test_actor1(test_state))

tensor([[ 0.0950, -0.3494]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[ 0.1185, -0.2919]], grad_fn=<AddmmBackward0>)


In [33]:
test_actor2= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_actor2', learning_rate=0.02)
print(test_actor2(test_state))
for target_param, param in zip(test_actor2.parameters(), test_actor1.parameters()):
    target_param.data.copy_(param.data)
print(test_actor2(test_state))

tensor([[ 0.1976, -0.0838]], grad_fn=<AddmmBackward0>)
tensor([[ 0.1185, -0.2919]], grad_fn=<AddmmBackward0>)
